In [ ]:
#pip install  'azureml-train-automl-runtime==1.41.0'

In [ ]:
import logging
import os
import tempfile

import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.script_run_config import ScriptRunConfig
from azureml.core.run import Run
from azureml.data.datapath import DataPath
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.model_proxy import ModelProxy

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
ws = Workspace.from_config()

# Choose an experiment name.
experiment_name = 'automl-iris'

experiment = Experiment ( ws , experiment_name )

output = {}
output [ 'Subscription ID' ] = ws.subscription_id
output [ 'Workspace Name' ] = ws.name
output [ 'Resource Group' ] = ws.resource_group
output [ 'Location' ] = ws.location
output [ 'Experiment Name' ] = experiment.name
output [ 'AMLS version' ] = azureml.core.VERSION

print ( output )

In [ ]:
train = pd.read_csv ( 'automl-iris-data/iris.csv' )
train.head ()

In [ ]:
data_train, data_test = train_test_split ( train , test_size=.2, stratify = train [ 'class' ] )
data_train, data_val = train_test_split ( data_train , test_size=.25, stratify = data_train [ 'class' ] )

blobstore_datadir = data_dir = 'automl-iris-data'  # Local directory to store data

if not os.path.isdir ( data_dir ) :
    os.mkdir ( data_dir )

train_data_fname = data_dir + '/train_data.csv'
val_data_fname = data_dir + '/val_data.csv'
test_data_fname = data_dir + '/test_data.csv'

data_train.to_csv ( train_data_fname , index = False )
data_val.to_csv ( val_data_fname , index = False )
data_test.to_csv ( test_data_fname , index = False )

datastore = ws.get_default_datastore ()
target = DataPath (
    datastore = datastore , path_on_datastore = blobstore_datadir , name = data_dir
)

Dataset.File.upload_directory(
    src_dir = data_dir , target = target , overwrite = True , show_progress = True
)

In [ ]:
train_dataset = Dataset.Tabular.from_delimited_files (
    path = [ ( datastore , blobstore_datadir + '/train_data.csv' ) ]
)

val_dataset = Dataset.Tabular.from_delimited_files (
    path = [ ( datastore , blobstore_datadir + '/val_data.csv' ) ]
)

test_dataset = Dataset.Tabular.from_delimited_files (
    path = [ ( datastore , blobstore_datadir + '/test_data.csv' ) ]
)

train_dataset = train_dataset.register (
    workspace = ws ,
    name = 'automl_iris_data_train' ,
    description = 'automl_iris_data_train' ,
    create_new_version = True ,
)

val_dataset = val_dataset.register (
    workspace = ws ,
    name = 'automl_iris_data_val' ,
    description = 'automl_iris_data_val' ,
    create_new_version = True ,
)

test_dataset = test_dataset.register (
    workspace = ws ,
    name = 'automl_iris_data_test' ,
    description = 'automl_iris_data_test' ,
    create_new_version = True ,
)

In [ ]:
compute_target = ComputeTarget ( workspace = ws , name = 'automl-training' )

In [ ]:
automl_settings = {
    'verbosity' : logging.INFO ,
}

automl_config = AutoMLConfig (
    task = 'classification' ,
    primary_metric = 'AUC_weighted' ,
    experiment_timeout_minutes = 30 ,
    debug_log= 'automl_errors.log' ,
    compute_target = compute_target ,
    training_data = train_dataset ,
    validation_data = val_dataset ,
    label_column_name = 'class' ,
    **automl_settings
)

In [ ]:
automl_run = experiment.submit ( automl_config , show_output = False )
_ = automl_run.wait_for_completion ( show_output = False )

In [ ]:
validation_metrics = automl_run.get_metrics ()
validation_metrics

In [ ]:
#automl_run = azureml.train.automl.run.AutoMLRun ( experiment , 'AutoML_55c202c5-e213-4d92-8190-7694e0986a5b' )
#automl_run

In [ ]:
best_run , best_model = automl_run.get_output ()
best_run

In [ ]:
#test_dataset = Dataset.get_by_name ( workspace = ws , name = 'automl_iris_data_test' )

In [ ]:
model_proxy = ModelProxy ( child_run = best_run , compute_target = compute_target )
predictions, metrics = model_proxy.test ( test_dataset ,  include_predictions_only = True )

In [ ]:
predictionsDF = predictions.to_pandas_dataframe ()
print ( predictionsDF )